# Settings

In [ ]:
import requests
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [3]:
pd.set_option('display.max_columns', 30)

In [398]:
#Connecting with database
conn = getConnection()

# Loading Data

In [13]:
data = requests.get("https://api.cartola.globo.com/clubes")
json = data.json()
data = []
for k in json:
    value = json[k]
    name = value["nome"]
    abb = value["abreviacao"]
    slug = value["slug"]
    surname = value["apelido"]
    fantasy_name = value["nome_fantasia"]
    id = value["id"]

    data.append([id, name, abb, slug, surname, fantasy_name])
header = ["Id", "Nome", "Abreviacao", "Slug", "Apelido", "Nome_Fantasia"]
df_clubes = pd.DataFrame(data, columns=header)

df_clubes

,Id,Nome,Abreviacao,Slug,Apelido,Nome_Fantasia
0,1,OUT,OUT,,,Outros
1,1349,IPA,IPA,,,Ipatinga
2,1371,CUI,CUI,,,Cuiabá
3,1390,ICA,ICA,,,Icasa
4,2190,OES,OES,,,Oeste
...,...,...,...,...,...,...
63,364,REM,REM,,,Remo
64,373,ACG,ACG,,,Atlético-GO
65,375,VIL,VIL,,,Vila Nova
66,386,SAO,SAO,,,São Caetano


In [ ]:
import requests
from config.db import getConnection


data = requests.get("https://api.cartola.globo.com/posicoes")
json = data.json()
rodada = json

conn = getConnection()
cursor = conn.cursor()


query = "INSERT INTO posicoes VALUES (%s, %s, %s)"

for item in json:
    id = json[item]["id"]
    nome = json[item]["nome"]
    abr = json[item]["abreviacao"]
    dados = [id, nome, abr]
    cursor.execute(query, dados)

conn.commit()
cursor.close()
conn.close()



C:\Users\Josef\AppData\Local\Temp\ipykernel_16812\1731277171.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  posicoes = pd.read_sql(query_posicoes, conn)


,posicao_id,nome,abreviacao
0,1,Goleiro,gol
1,2,Lateral,lat
2,3,Zagueiro,zag
3,4,Meia,mei
4,5,Atacante,ata


In [401]:
query_scouts = "SELECT * FROM cartola.scouts"
scouts = pd.read_sql(query_scouts, conn)
scouts.head()

C:\Users\Josef\AppData\Local\Temp\ipykernel_16812\2560286167.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  scouts = pd.read_sql(query_scouts, conn)


,id,nome,abreviacao,pontos,observacao
0,1,Gol,G,8.0,
1,2,Assistência,A,5.0,
2,3,Finalização na trave,FT,3.0,
3,4,Finalização defendida,FD,1.2,
4,5,Finalização pra fora,FF,0.8,


In [402]:
query_rodadas = "SELECT * FROM cartola.rodadas"
rodadas = pd.read_sql(query_rodadas, conn)
rodadas.head()

C:\Users\Josef\AppData\Local\Temp\ipykernel_16812\2058869999.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rodadas = pd.read_sql(query_rodadas, conn)


,rodada_id,nome_rodada,inicio,fim
0,1,Rodada 1,2025-03-29 18:30:00,2025-03-31 20:00:00
1,2,Rodada 2,2025-04-05 18:30:00,2025-04-06 20:30:00
2,3,Rodada 3,2025-04-12 16:00:00,2025-04-13 20:30:00
3,4,Rodada 4,2025-04-16 18:30:00,2025-04-17 21:30:00
4,5,Rodada 5,2025-04-19 16:00:00,2025-04-21 20:00:00


In [403]:
query_status = "SELECT * FROM cartola.status"
status = pd.read_sql(query_status, conn)
status.head()

C:\Users\Josef\AppData\Local\Temp\ipykernel_16812\486490424.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  status = pd.read_sql(query_status, conn)


,status_id,nome
0,2,Dúvida
1,3,Suspenso
2,5,Contundido
3,6,Nulo
4,7,Provável


In [404]:
query_partidas_historico = "SELECT * FROM cartola.partidas_historico"
partidas_historico = pd.read_sql(query_partidas_historico, conn)
partidas_historico.head()

C:\Users\Josef\AppData\Local\Temp\ipykernel_16812\4142430073.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  partidas_historico = pd.read_sql(query_partidas_historico, conn)


,id,local,clube_mandante_id,clube_visitante_id,placar_mandante,placar_visitante,clube_mandante_posicao,clube_visitante_posicao,aproveitamento_mandante,aproveitamento_visitante,rodada_id
0,1,Morumbis,276,292,0,0,11,20,"['e', 'e', 'v', 'e', 'e']","['d', 'd', 'd', 'e', 'd']",1
1,2,Mineirão,283,2305,2,1,4,16,"['e', 'v', 'd', 'v', 'v']","['e', 'v', 'e', 'e', 'd']",1
2,3,Arena do Grêmio,284,282,2,1,13,10,"['d', 'd', 'e', 'e', 'v']","['e', 'd', 'v', 'e', 'v']",1
3,4,Castelão (CE),356,266,2,0,17,5,"['e', 'd', 'd', 'e', 'e']","['v', 'v', 'e', 'd', 'v']",1
4,5,Alfredo Jaconi,286,287,2,0,15,18,"['v', 'd', 'e', 'd', 'd']","['e', 'v', 'e', 'e', 'd']",1


In [405]:
query_partidas_totais = "SELECT * FROM cartola.partidas_totais"
partidas_totais = pd.read_sql(query_partidas_totais, conn)
partidas_totais.head()

C:\Users\Josef\AppData\Local\Temp\ipykernel_16812\3328968777.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  partidas_totais = pd.read_sql(query_partidas_totais, conn)


,id,mandante_id,visitante_id,rodada_id
0,1,262,285,1
1,2,267,277,1
2,3,275,263,1
3,4,276,292,1
4,5,280,354,1


In [406]:
query_atleta_pontuacao = "SELECT * FROM cartola.atleta_pontuacao"
atleta_pontuacao = pd.read_sql(query_atleta_pontuacao, conn)
atleta_pontuacao.head()

C:\Users\Josef\AppData\Local\Temp\ipykernel_16812\3859137925.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  atleta_pontuacao = pd.read_sql(query_atleta_pontuacao, conn)


,pontuacao_id,apelido,atleta_id,pontuacao,posicao_id,clube_id,rodada_id,entrou_em_campo,scouts
0,1,Yuri Alberto,100652,2.9,5,264,1,1,"{""FD"": 2, ""FS"": 1}"
1,2,Matheus Henrique,100742,-1.9,4,283,1,1,"{""CA"": 1, ""FC"": 3}"
2,3,Lucas Esteves,100846,2.8,2,284,1,1,"{""DS"": 1, ""FF"": 1, ""FS"": 1}"
3,4,Vitinho,100898,10.7,2,263,1,1,"{""DS"": 4, ""FC"": 1, ""SG"": 1}"
4,5,Ronald Lopes,100969,-1.1,4,287,1,1,"{""CA"": 1, ""FC"": 2, ""FS"": 1}"


# Adjusting partidas

In [407]:
partidas_historico = partidas_historico.drop(columns=["id", "local", "rodada_id"])

In [408]:
partidas_historico["key"] = partidas_historico["clube_mandante_id"].astype(str) + "_" + partidas_historico["clube_visitante_id"].astype(str)
partidas_historico


,clube_mandante_id,clube_visitante_id,placar_mandante,placar_visitante,clube_mandante_posicao,clube_visitante_posicao,aproveitamento_mandante,aproveitamento_visitante,key
0,276,292,0,0,11,20,"['e', 'e', 'v', 'e', 'e']","['d', 'd', 'd', 'e', 'd']",276_292
1,283,2305,2,1,4,16,"['e', 'v', 'd', 'v', 'v']","['e', 'v', 'e', 'e', 'd']",283_2305
2,284,282,2,1,13,10,"['d', 'd', 'e', 'e', 'v']","['e', 'd', 'v', 'e', 'v']",284_282
3,356,266,2,0,17,5,"['e', 'd', 'd', 'e', 'e']","['v', 'v', 'e', 'd', 'v']",356_266
4,286,287,2,0,15,18,"['v', 'd', 'e', 'd', 'd']","['e', 'v', 'e', 'e', 'd']",286_287
...,...,...,...,...,...,...,...,...,...
65,267,275,0,1,14,1,"['v', 'd', 'e', 'd', 'd']","['v', 'v', 'v', 'd', 'v']",267_275
66,284,277,1,0,13,19,"['d', 'd', 'e', 'e', 'v']","['d', 'v', 'd', 'd', 'd']",284_277
67,283,262,2,1,4,3,"['e', 'v', 'd', 'v', 'v']","['v', 'v', 'e', 'v', 'd']",283_262
68,280,2305,1,0,2,16,"['v', 'v', 'v', 'v', 'v']","['e', 'v', 'e', 'e', 'd']",280_2305


In [409]:
partidas_totais["key"] = partidas_totais["mandante_id"].astype(str) + "_" + partidas_totais["visitante_id"].astype(str)
partidas_totais

,id,mandante_id,visitante_id,rodada_id,key
0,1,262,285,1,262_285
1,2,267,277,1,267_277
2,3,275,263,1,275_263
3,4,276,292,1,276_292
4,5,280,354,1,280_354
...,...,...,...,...,...
375,376,282,267,38,282_267
376,377,285,280,38,285_280
377,378,287,276,38,287_276
378,379,354,275,38,354_275


In [410]:
dados_partida = pd.merge(partidas_totais, partidas_historico, on="key")

In [411]:
dados_partida = dados_partida.drop(columns=["key", "clube_mandante_id", "clube_visitante_id"])

In [412]:
dados_partida["resultado"] = dados_partida.apply(lambda x: 1 if x["placar_mandante"] > x["placar_visitante"] else (0 if x["placar_mandante"] == x["placar_visitante"] else -1), axis=1)

In [ ]:
def getChanceToWin(pos1, pos2, option):
    chances = {pos1: 0, pos2: 0}

    for k, v in chances.items():
        if k <= 10: 
            chances[k] = 3 # Ponto por estar no meio da tabela
        elif k <= 15:
            chances[k] = 2 # Ponto por estar na 2 metade da tabela
        else: 
            chances[k] = 1 # Ponto por estar entre os 
    
    chances[pos1] += 1 # Ganha ponto por ser mandante

    if option == 1:
        dif = chances[pos1] - chances[pos2]
    else:
        dif = chances[pos2] - chances[pos1]

    if dif >= 1:
        return 1
    elif dif <= -1:
        return -1
    else:
        return 0
    

dados_partida["dificuldade_mandante"] = dados_partida.apply(lambda df: getChanceToWin(df["clube_mandante_posicao"], df["clube_visitante_posicao"], 1), axis=1)
dados_partida["dificuldade_visitante"] = dados_partida.apply(lambda df: getChanceToWin(df["clube_mandante_posicao"], df["clube_visitante_posicao"], 2), axis=1)

dados_partida


,id,mandante_id,visitante_id,rodada_id,placar_mandante,placar_visitante,clube_mandante_posicao,clube_visitante_posicao,aproveitamento_mandante,aproveitamento_visitante,resultado,dificuldade_mandante,dificuldade_visitante
0,1,262,285,1,1,1,3,9,"['v', 'v', 'e', 'v', 'd']","['e', 'd', 'e', 'v', 'd']",0,1,-1
1,2,267,277,1,2,1,14,19,"['v', 'd', 'e', 'd', 'd']","['d', 'v', 'd', 'd', 'd']",1,1,-1
2,3,275,263,1,0,0,1,12,"['v', 'v', 'v', 'd', 'v']","['d', 'e', 'd', 'v', 'd']",0,1,-1
3,4,276,292,1,0,0,11,20,"['e', 'e', 'v', 'e', 'e']","['d', 'd', 'd', 'e', 'd']",0,1,-1
4,5,280,354,1,2,2,2,7,"['v', 'v', 'v', 'v', 'v']","['d', 'v', 'd', 'e', 'v']",0,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,66,283,262,7,2,1,4,3,"['e', 'v', 'd', 'v', 'v']","['v', 'v', 'e', 'v', 'd']",1,1,-1
66,67,284,277,7,1,0,13,19,"['d', 'd', 'e', 'e', 'v']","['d', 'v', 'd', 'd', 'd']",1,1,-1
67,68,265,263,7,1,0,6,12,"['e', 'd', 'v', 'v', 'v']","['d', 'e', 'd', 'v', 'd']",1,1,-1
68,69,354,287,7,1,0,7,18,"['d', 'v', 'd', 'e', 'v']","['e', 'v', 'e', 'e', 'd']",1,1,-1


In [414]:
mandante_df = dados_partida.rename(columns={
    'mandante_id': 'clube_id',
    'visitante_id': 'rival_id',
    'placar_mandante': 'placar',
    'clube_mandante_posicao': 'posicao',
    'aproveitamento_mandante': 'aproveitamento',
    'dificuldade_mandante': 'dificuldade'
})[['clube_id', 'rival_id', 'rodada_id', 'placar', 'posicao', 'aproveitamento', 'dificuldade']]

# Depois: a linha do VISITANTE
visitante_df = dados_partida.rename(columns={
    'visitante_id': 'clube_id',
    'mandante_id': 'rival_id',
    'placar_visitante': 'placar',
    'clube_visitante_posicao': 'posicao',
    'aproveitamento_visitante': 'aproveitamento',
    'dificuldade_visitante': 'dificuldade'
})[['clube_id', 'rival_id', 'rodada_id', 'placar', 'posicao', 'aproveitamento', 'dificuldade']]

# Junta os dois
partidas_formatadas = pd.concat([mandante_df, visitante_df], ignore_index=True)
partidas_formatadas.head(20)

,clube_id,rival_id,rodada_id,placar,posicao,aproveitamento,dificuldade
0,262,285,1,1,3,"['v', 'v', 'e', 'v', 'd']",1
1,267,277,1,2,14,"['v', 'd', 'e', 'd', 'd']",1
2,275,263,1,0,1,"['v', 'v', 'v', 'd', 'v']",1
3,276,292,1,0,11,"['e', 'e', 'v', 'e', 'e']",1
4,280,354,1,2,2,"['v', 'v', 'v', 'v', 'v']",1
5,283,2305,1,2,4,"['e', 'v', 'd', 'v', 'v']",1
6,284,282,1,2,13,"['d', 'd', 'e', 'e', 'v']",0
7,265,264,1,1,6,"['e', 'd', 'v', 'v', 'v']",1
8,356,266,1,2,17,"['e', 'd', 'd', 'e', 'e']",-1
9,286,287,1,2,15,"['v', 'd', 'e', 'd', 'd']",1


# Adjusting atleta_pontuacao

In [415]:
import json

# Expande os scouts em colunas separadas
scouts_df = atleta_pontuacao['scouts'].apply(lambda x: json.loads(x) if isinstance(x, str) else {})
scouts_df = pd.json_normalize(scouts_df)

# Junta com o DataFrame principal
dados = pd.concat([atleta_pontuacao.drop(columns=['scouts']), scouts_df], axis=1)

In [416]:
dados = dados.merge(posicoes, on='posicao_id', how='left')
dados = dados.merge(clubes, on='clube_id', how='left')
dados.merge
dados.head()

,pontuacao_id,apelido_x,atleta_id,pontuacao,posicao_id,clube_id,rodada_id,entrou_em_campo,FD,FS,CA,FC,DS,FF,SG,...,V,PC,DP,CV,FT,PP,PS,GC,nome_x,abreviacao_x,nome_y,abreviacao_y,slug,apelido_y,nome_fantasia
0,1,Yuri Alberto,100652,2.9,5,264,1,1,2.0,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Atacante,ata,COR,COR,corinthians,Timão,Corinthians
1,2,Matheus Henrique,100742,-1.9,4,283,1,1,NaN,NaN,1.0,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Meia,mei,CRU,CRU,cruzeiro,Raposa,Cruzeiro
2,3,Lucas Esteves,100846,2.8,2,284,1,1,NaN,1.0,NaN,NaN,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lateral,lat,GRE,GRE,gremio,Imortal,Grêmio
3,4,Vitinho,100898,10.7,2,263,1,1,NaN,NaN,NaN,1.0,4.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lateral,lat,BOT,BOT,botafogo,Glorioso,Botafogo
4,5,Ronald Lopes,100969,-1.1,4,287,1,1,NaN,1.0,1.0,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Meia,mei,VIT,VIT,vitoria,Vitória,Vitória


In [417]:
dados.isnull().sum()
dados = dados.fillna(0)
dados

,pontuacao_id,apelido_x,atleta_id,pontuacao,posicao_id,clube_id,rodada_id,entrou_em_campo,FD,FS,CA,FC,DS,FF,SG,...,V,PC,DP,CV,FT,PP,PS,GC,nome_x,abreviacao_x,nome_y,abreviacao_y,slug,apelido_y,nome_fantasia
0,1,Yuri Alberto,100652,2.9,5,264,1,1,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atacante,ata,COR,COR,corinthians,Timão,Corinthians
1,2,Matheus Henrique,100742,-1.9,4,283,1,1,0.0,0.0,1.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Meia,mei,CRU,CRU,cruzeiro,Raposa,Cruzeiro
2,3,Lucas Esteves,100846,2.8,2,284,1,1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lateral,lat,GRE,GRE,gremio,Imortal,Grêmio
3,4,Vitinho,100898,10.7,2,263,1,1,0.0,0.0,0.0,1.0,4.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lateral,lat,BOT,BOT,botafogo,Glorioso,Botafogo
4,5,Ronald Lopes,100969,-1.1,4,287,1,1,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Meia,mei,VIT,VIT,vitoria,Vitória,Vitória
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,2298,Luis Zubeldía,99366,0.0,6,276,7,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Técnico,tec,SAO,SAO,sao-paulo,Tricolor,São Paulo
2298,2299,Michael,99392,1.5,5,262,7,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atacante,ata,FLA,FLA,flamengo,Mengão,Flamengo
2299,2300,Rossi,99552,15.4,1,262,7,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Goleiro,gol,FLA,FLA,flamengo,Mengão,Flamengo
2300,2301,Arias,99815,14.5,4,266,7,1,0.0,2.0,0.0,1.0,2.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Meia,mei,FLU,FLU,fluminense,Tricolor,Fluminense


In [418]:
dados = dados.merge(
    partidas_formatadas,
    left_on=['rodada_id', 'clube_id'],
    right_on=['rodada_id', 'clube_id'],
    how='left'
)
dados

,pontuacao_id,apelido_x,atleta_id,pontuacao,posicao_id,clube_id,rodada_id,entrou_em_campo,FD,FS,CA,FC,DS,FF,SG,...,PP,PS,GC,nome_x,abreviacao_x,nome_y,abreviacao_y,slug,apelido_y,nome_fantasia,rival_id,placar,posicao,aproveitamento,dificuldade
0,1,Yuri Alberto,100652,2.9,5,264,1,1,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,Atacante,ata,COR,COR,corinthians,Timão,Corinthians,265,1,8,"['d', 'd', 'v', 'd', 'v']",-1
1,2,Matheus Henrique,100742,-1.9,4,283,1,1,0.0,0.0,1.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,Meia,mei,CRU,CRU,cruzeiro,Raposa,Cruzeiro,2305,2,4,"['e', 'v', 'd', 'v', 'v']",1
2,3,Lucas Esteves,100846,2.8,2,284,1,1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,Lateral,lat,GRE,GRE,gremio,Imortal,Grêmio,282,2,13,"['d', 'd', 'e', 'e', 'v']",0
3,4,Vitinho,100898,10.7,2,263,1,1,0.0,0.0,0.0,1.0,4.0,0.0,1.0,...,0.0,0.0,0.0,Lateral,lat,BOT,BOT,botafogo,Glorioso,Botafogo,275,0,12,"['d', 'e', 'd', 'v', 'd']",-1
4,5,Ronald Lopes,100969,-1.1,4,287,1,1,0.0,1.0,1.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,Meia,mei,VIT,VIT,vitoria,Vitória,Vitória,286,0,18,"['e', 'v', 'e', 'e', 'd']",-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,2298,Luis Zubeldía,99366,0.0,6,276,7,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,Técnico,tec,SAO,SAO,sao-paulo,Tricolor,São Paulo,356,0,11,"['e', 'e', 'v', 'e', 'e']",1
2298,2299,Michael,99392,1.5,5,262,7,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,Atacante,ata,FLA,FLA,flamengo,Mengão,Flamengo,283,1,3,"['v', 'v', 'e', 'v', 'd']",-1
2299,2300,Rossi,99552,15.4,1,262,7,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,Goleiro,gol,FLA,FLA,flamengo,Mengão,Flamengo,283,1,3,"['v', 'v', 'e', 'v', 'd']",-1
2300,2301,Arias,99815,14.5,4,266,7,1,0.0,2.0,0.0,1.0,2.0,1.0,0.0,...,0.0,0.0,0.0,Meia,mei,FLU,FLU,fluminense,Tricolor,Fluminense,292,2,5,"['v', 'v', 'e', 'd', 'v']",1


In [419]:
dados = dados.drop(columns=["abreviacao_x", "abreviacao_y", "slug", "apelido_y", "nome_fantasia", "entrou_em_campo", 'pontuacao_id', 'atleta_id', 'posicao_id', 'clube_id', 'rival_id', 'placar', 'posicao', 'aproveitamento'])
dados = dados.rename({"nome_x": "posicao", "nome_y": "time", "apelido_x": "Nome"}, axis=1)
dados

,Nome,pontuacao,rodada_id,FD,FS,CA,FC,DS,FF,SG,A,G,I,DE,GS,V,PC,DP,CV,FT,PP,PS,GC,posicao,time,dificuldade
0,Yuri Alberto,2.9,1,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atacante,COR,-1
1,Matheus Henrique,-1.9,1,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Meia,CRU,1
2,Lucas Esteves,2.8,1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lateral,GRE,0
3,Vitinho,10.7,1,0.0,0.0,0.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lateral,BOT,-1
4,Ronald Lopes,-1.1,1,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Meia,VIT,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,Luis Zubeldía,0.0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Técnico,SAO,1
2298,Michael,1.5,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atacante,FLA,-1
2299,Rossi,15.4,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Goleiro,FLA,-1
2300,Arias,14.5,7,0.0,2.0,0.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Meia,FLU,1


In [420]:
dados['mitou'] = dados['pontuacao'] > 7.27
dados

,Nome,pontuacao,rodada_id,FD,FS,CA,FC,DS,FF,SG,A,G,I,DE,GS,V,PC,DP,CV,FT,PP,PS,GC,posicao,time,dificuldade,mitou
0,Yuri Alberto,2.9,1,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atacante,COR,-1,False
1,Matheus Henrique,-1.9,1,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Meia,CRU,1,False
2,Lucas Esteves,2.8,1,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lateral,GRE,0,False
3,Vitinho,10.7,1,0.0,0.0,0.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Lateral,BOT,-1,True
4,Ronald Lopes,-1.1,1,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Meia,VIT,-1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2297,Luis Zubeldía,0.0,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Técnico,SAO,1,False
2298,Michael,1.5,7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atacante,FLA,-1,False
2299,Rossi,15.4,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Goleiro,FLA,-1,True
2300,Arias,14.5,7,0.0,2.0,0.0,1.0,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Meia,FLU,1,True


NameError: name 'dados' is not defined